# 1. predicting adsorption for each adsorption data point using Vt&BET only

In [32]:
import pandas as pd
import numpy as np
import os 
BET_path = 'C:/Kai_Zhang/MachineLearning/Unified gas Adsorption/Treated/CO2'

def pred_dataset(file_names):
    source_path = 'C:/Kai_Zhang/MachineLearning/Unified gas Adsorption/CO2_adsorption/new_data'
    train_df = pd.DataFrame()
    for file_name in file_names:
        temp_data = pd.read_excel(os.path.join(source_path,file_name+'-04-19-2022.xlsx'),skiprows= 1 )
        #temp_data = temp_data.dropna(axis=0,how = 'any',subset = feature_set)
        train_df = pd.concat([train_df,temp_data],axis=0)
       
    return train_df

In [97]:
from scipy import interpolate
import numpy as np
import pandas as pd

def inter_data(x_old,y_old):
    
    f = interpolate.interp1d(x_old, y_old,'nearest-up',bounds_error=False) 

    
    """seg1 = np.linspace(min(x_old),min(x_old)+0.2,num=20)
    seg2 = np.linspace(min(x_old)+0.2,max(x_old)-0.2,num=12)
    seg3 = np.linspace(max(x_old)-0.2, max(x_old),num=20)
    x_new = np.append(seg1,seg2[1:-1])
    x_new = np.append(x_new,seg3)  """ 
    x_new = np.linspace(min(x_old), 0.99,num=35)
       
    """
        for ele in x_old:
            t = bi_find(x_new,ele)

            x_new[t] = ele
        
        if (len(x_new)>150):
            x_new = x_new[len(x_new)-150:]
         = np.arange(min(x_old), max(x_old), (max(x_old)-min(x_old))/150)
    """
    
    y_new = f(x_new)
    #X_feature = np.append(x_new.reshape(-1,1),y_new.reshape(-1,1),axis=1).reshape(2,35).tolist()
    #x_mat = np.append(x_mat,X_feature,axis=1)
       
    return x_new, y_new

In [98]:
data = pred_dataset(["CO2"])
data

,Unnamed: 0,Literature,Lines,GAC,GAS,E,S,A,B,V,...,%N,Temp(K),Pressure,Adsorp(mmol/g),Index,logP,logQ,logD,Label,Vmic_index
0,0,1.0,0,UC-600-2,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,1.98,298.0,0.048715,0.547018,0,-3.021763,-0.603274,2.418489,CO2,0
1,1,1.0,1,UC-600-2,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,1.98,298.0,0.104147,0.935902,0,-2.261948,-0.066244,2.195704,CO2,0
2,2,1.0,2,UC-600-2,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,1.98,298.0,0.137430,1.095365,0,-1.984640,0.091087,2.075727,CO2,0
3,3,1.0,3,UC-600-2,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,1.98,298.0,0.208412,1.502384,0,-1.568239,0.407053,1.975292,CO2,0
4,4,1.0,4,UC-600-2,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,1.98,298.0,0.266122,1.715469,0,-1.323800,0.539687,1.863487,CO2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23336,23431,NaN,35,NaN,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,NaN,298.0,2.962963,4.588613,2190,1.086190,1.523578,0.437388,CO2,0
23337,23432,NaN,36,NaN,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,NaN,298.0,4.027778,5.245077,2190,1.393215,1.657290,0.264075,CO2,0
23338,23433,NaN,37,NaN,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,NaN,298.0,5.972222,6.029895,2190,1.787119,1.796730,0.009610,CO2,0
23339,23434,NaN,38,NaN,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,NaN,298.0,7.939815,6.638302,2190,2.071890,1.892856,-0.179034,CO2,0


In [99]:
BET_df = pd.DataFrame()
cols = ["E","S","A","B","V","BET","Vt","Vmic","Vmeso","Temp(K)","Pressure","Adsorp(mmol/g)","Index"]
def bet_extraction(temp_df:pd.DataFrame):
    global BET_df
    literature = temp_df["Literature"].values[0]
    gac_name = temp_df["GAC"].values[0]
    print(literature,gac_name)
    try:
        if literature!=np.nan:
                bet_data = BET_path+'/'+str(int(literature))+"/"+gac_name+".csv"
                #print(bet_data)
                bet_curve = pd.read_csv(bet_data,header=None)
                x = bet_curve.iloc[:,0].values.tolist()
                y = bet_curve.iloc[:,1].values.tolist()
                _,new_y = inter_data(x,y)
                rows = len(temp_df)
                bet_matrix = [new_y for _ in range(rows)]
                bet_df = pd.DataFrame(bet_matrix,columns = ["BET_"+str(i) for i in range(len(new_y))])
                bet_df = pd.concat([bet_df,temp_df[cols].reset_index()],axis= 1,ignore_index=True)
                print(bet_df.shape)
                BET_df = pd.concat([BET_df,bet_df,],axis= 0,ignore_index=True)    

        #else:
        #    print("There is not leterature for this gac: {}".format(gac_name))         
    except:
        print("The BET curve for gac: {} from paper {} is not available.".format(gac_name,literature))
total_index = list(set(data["Index"].values.tolist()))
for index in total_index:
    temp_df =  data[data["Index"]==index]
    bet_extraction(temp_df)

1.0 UC-600-2
(16, 49)
1.0 UC-600-3
(17, 49)
1.0 UC-600-4
(21, 49)
1.0 UC-600-2
(21, 49)
1.0 UC-600-3
(19, 49)
1.0 UC-600-4
(22, 49)
1.0 UC-650-2
(21, 49)
1.0 UC-650-3
(17, 49)
1.0 UC-650-4
(18, 49)
1.0 UC-650-2
(16, 49)
1.0 UC-650-3
(18, 49)
1.0 UC-650-4
(17, 49)
1.0 UC-700-2
(15, 49)
1.0 UC-700-3
(17, 49)
1.0 UC-700-4
(16, 49)
1.0 UC-700-2
(14, 49)
1.0 UC-700-3
(20, 49)
1.0 UC-700-4
(19, 49)
2.0 CN-600-3
(22, 49)
2.0 CN-600-2
(16, 49)
2.0 CN-600-4
(21, 49)
2.0 CN-700-2
(20, 49)
2.0 CN-700-3
(20, 49)
2.0 CN-800-2
(21, 49)
2.0 CN-800-3
(21, 49)
2.0 CN-600-2
(14, 49)
2.0 CN-600-3
(12, 49)
2.0 CN-600-4
(12, 49)
2.0 CN-700-2
(16, 49)
2.0 CN-700-3
(11, 49)
2.0 CN-800-3
(15, 49)
3.0 PC-2
(12, 49)
3.0 PC-1_4
(11, 49)
3.0 PC-1_5
(10, 49)
4.0 c-CBAP-1N
(16, 49)
4.0 c-CBAP-1N
(13, 49)
4.0 c-CBAP-1N
(13, 49)
4.0 c-CBAP-1N
(11, 49)
5.0 AC
(7, 49)
5.0 AC
(7, 49)
5.0 AC
(7, 49)
6.0 O-3-650-110
The BET curve for gac: O-3-650-110 from paper 6.0 is not available.
6.0 O-3-650-110
The BET curve for gac: 

In [100]:
BET_df.to_csv('./CO2_Adding_bet_35_round_end.csv')

In [101]:
BET_df

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,283.382789,296.735905,299.703264,302.670623,304.154303,304.154303,304.154303,304.154303,305.637982,305.637982,...,0.1,0.2809,1239.0,0.430,NaN,NaN,298.0,0.048715,0.547018,0
1,283.382789,296.735905,299.703264,302.670623,304.154303,304.154303,304.154303,304.154303,305.637982,305.637982,...,0.1,0.2809,1239.0,0.430,NaN,NaN,298.0,0.104147,0.935902,0
2,283.382789,296.735905,299.703264,302.670623,304.154303,304.154303,304.154303,304.154303,305.637982,305.637982,...,0.1,0.2809,1239.0,0.430,NaN,NaN,298.0,0.137430,1.095365,0
3,283.382789,296.735905,299.703264,302.670623,304.154303,304.154303,304.154303,304.154303,305.637982,305.637982,...,0.1,0.2809,1239.0,0.430,NaN,NaN,298.0,0.208412,1.502384,0
4,283.382789,296.735905,299.703264,302.670623,304.154303,304.154303,304.154303,304.154303,305.637982,305.637982,...,0.1,0.2809,1239.0,0.430,NaN,NaN,298.0,0.266122,1.715469,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16468,69.176471,283.294118,319.529412,319.529412,349.176471,349.176471,368.941176,388.705882,388.705882,388.705882,...,0.1,0.2809,1052.0,0.651,0.4,0.251,298.0,0.362518,1.094301,1900
16469,69.176471,283.294118,319.529412,319.529412,349.176471,349.176471,368.941176,388.705882,388.705882,388.705882,...,0.1,0.2809,1052.0,0.651,0.4,0.251,298.0,0.551223,1.393736,1900
16470,69.176471,283.294118,319.529412,319.529412,349.176471,349.176471,368.941176,388.705882,388.705882,388.705882,...,0.1,0.2809,1052.0,0.651,0.4,0.251,298.0,0.700574,1.604632,1900
16471,69.176471,283.294118,319.529412,319.529412,349.176471,349.176471,368.941176,388.705882,388.705882,388.705882,...,0.1,0.2809,1052.0,0.651,0.4,0.251,298.0,0.834203,1.793328,1900


In [44]:
BET_df = pd.read_csv("Methane_Adding_bet_35_new.csv")
BET_df.dropna(axis=0,how="any",subset = ["BET","Vt","Vmic","BET_34"],inplace=True)
#BET_df.dropna(axis=0,how="any",subset = ["BET_34"],inplace=True)

# Using evenly selected BET cureve to find the feature importance of eachi region in the BET curve by the interval of 0.1

In [68]:
# data splitting
#from random import shuffle
import pandas as pd
from sklearn.model_selection import train_test_split
from numpy import random
def group_split(BET_df:pd.DataFrame,random_state = 42):
    #BET_df = BET_df[BET_df['Pressure']>3]
    indexs = list(set(BET_df["Index"].values.tolist()))
    #shuffle(indexs)
    _,test_index = train_test_split(indexs,test_size= 0.2,random_state =random_state)

    #test_index = random.choice(indexs,len(indexs)//5,replace=False).tolist()
    train_df = BET_df[~BET_df["Index"].isin(test_index)]
    test_df = BET_df[BET_df["Index"].isin(test_index)]
    return train_df,test_df

In [69]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor,\
    BaggingRegressor,ExtraTreesRegressor,RandomForestRegressor
from lightgbm import LGBMRegressor  
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
  
n_estimators = [50,100,120,150,180,200,300]

# define different models#,
models = [
    #('SVR',SVR(max_iter=100000)),
    #('DT',DecisionTreeRegressor(random_state=42)),\
    # ('ADBR',AdaBoostRegressor(random_state=42)), 
    #("GBR",GradientBoostingRegressor(random_state=42)),\
    #('BG',BaggingRegressor(random_state=42,n_jobs=-1)),
    #('ETR',ExtraTreesRegressor(random_state=42,n_jobs=-1)),\
    #('RF',RandomForestRegressor(n_jobs=-1,random_state=42)),
    ('LGBM',LGBMRegressor(n_jobs = -1,random_state = 42)),\
    #('MLPR',MLPRegressor(random_state=42))
    #('BGLGBM',BaggingRegressor(LGBMRegressor(n_estimators = 200, n_jobs = -1,random_state = 42), random_state=42,n_jobs=-1)),\
    #('BGETR',BaggingRegressor(ExtraTreesRegressor(n_estimators = 180,random_state=42,n_jobs=6),random_state=42,n_jobs=-1))
    ]

# set search parameters grid for different models
para_grids = { 
    'SVR':{'kernel':['linear','poly','rbf','sigmoid','precomputed']},\
    'DT':{'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson']},\
    'ADBR':{'n_estimators':n_estimators,'learning_rate':[0.1,0.5,1,2],'loss':['linear','square','exponential']},\
    'GBR':{'n_estimators':n_estimators,'learning_rate':[0.1,0.5,1,2]},\
    'BG':{'n_estimators':[10,50,100]},\
    'ETR':{'n_estimators':n_estimators},\
    'RF':{'n_estimators':n_estimators},\
    'LGBM':{'num_leaves':[10,20,30,50,],'learning_rate': [0.05,0.1,0.5,1],'n_estimators':n_estimators},\
    'BGLGBM':{'n_estimators':[10,30,50]},\
    'BGETR':{'n_estimators':[10]},\
    'MLPR':{'hidden_layer_sizes':[(4,100),(5,100),(6,100)],
            "activation":["logistic","relu"],\
            "solver":['sgd', 'adam']}
    }

In [94]:
from sklearn.model_selection import GridSearchCV,cross_validate,GroupKFold
from sklearn.ensemble import ExtraTreesRegressor
from  sklearn.metrics import mean_squared_error,r2_score
from sklearn.utils import shuffle
from sklearn.inspection import permutation_importance

def model_CV(train_x,train_y,groups,model,para_grid):

    out_cv = GroupKFold(n_splits = 5)
    result = GridSearchCV(model,para_grid,cv= out_cv.get_n_splits(groups =groups),
    scoring='neg_mean_squared_error', return_train_score=True,n_jobs=-1)
    result.fit(train_x,train_y)
    
    model_refit =model.set_params(**result.best_params_)
    train_cv = cross_validate(model_refit,train_x,train_y,groups = groups,cv =out_cv,scoring = ('r2', 'neg_mean_squared_error'))
    train_mse_cv = -train_cv['test_neg_mean_squared_error'].mean()
    train_r2_cv = train_cv['test_r2'].mean()
    
    return [train_r2_cv,train_mse_cv],result.best_params_

# model evaluation
def model_eval(model,test_x,test_y):
      
    test_pre = model.predict(test_x)
    test_r2 = r2_score(test_pre,test_y)
    test_mse = mean_squared_error(test_y,test_pre)
    return test_r2,test_mse

# comparing different models
def model_comparison(model_list,para_grids,feature_list,gas_list):
    gas_list = gas_list 
    input_feature = feature_list
    output = ['Adsorp(mmol/g)']
    result_total = []
    for gas in gas_list:
        if gas =='total':
            train_df_com = train_df
            test_df_com = test_df
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            groups = train_df_com['Index'].values
            train_x, train_y, groups = shuffle(train_x, train_y, groups, random_state=42)
            
            for model_name, model in model_list:
                result, best_param = model_CV(train_x,train_y.squeeze(),groups,model,para_grids[model_name])
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())
                test_r2_total,test_mse_total = model_eval(model_refit,test_x,test_y.squeeze()) 
                for gases in gas_list[1:]:
                    test_df_com = test_df[test_df['Label']==gases]
                    test_xs = test_df_com[input_feature]
                    test_ys = test_df_com[output].values
                    test_r2,test_mse = model_eval(model_refit,test_xs,test_ys.squeeze()) 
                    result_total.append([gases,model_name+'_total',result[0],result[1],test_r2_total,test_mse_total,test_r2,test_mse,best_param])
                    print('Dataset {}, Algorithm {}, Test_r2 {}, Test_error {}'.format(gas,model_name+'_total',test_r2,test_mse))    
        else:
            train_df_com = train_df[train_df['Label']==gas]
            test_df_com = test_df[test_df['Label']==gas]
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            groups = train_df_com['Index']
            train_x, train_y, groups = shuffle(train_x, train_y, groups, random_state=42)
            for model_name, model in model_list:
                result, best_param = model_CV(train_x,train_y.squeeze(),groups,model,para_grids[model_name])
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())

                feature_importances = permutation_importance(model, train_x, train_y, n_repeats=30,random_state=0) 
                
                #feature_importances = model_refit.feature_importances_.tolist()  #####new lines added to record the feature importance of each round
                test_r2,test_mse = model_eval(model_refit,test_x,test_y.squeeze()) 
                result_total.append([gas,model_name+'_separate',result[0],result[1],-1,-1, test_r2,test_mse,best_param]+feature_importances.importances_mean.tolist()) # adding feature importance valuels to the end of outputted results
                #result_total.append([gas,model_name+'_separate',result[0],result[1],-1,-1, test_r2,test_mse,best_param]+feature_importances)
                print('Dataset {}, Algorithm {}, Test_r2 {}, Test_error {}'.format(gas,model_name+'_total',test_r2,test_mse))     
    return result_total

In [103]:
BET_df = pd.read_csv("CO2_Adding_bet_35_round_end.csv")
BET_df.dropna(axis=0,how="any",subset = ["BET","Vt","Vmic","BET_34"],inplace=True)

In [104]:
import os
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
#from random import shuffle
conditions = ['S','V','Temp(K)','Pressure']
bets =  ["BET_"+str(i) for i in range(33,-1,-1)]
#bets = bets[:10][::-1]+bets[10:]

print(bets)
base_feature = conditions+bets
#feature_list = [base_feature,base_feature+["BET"],base_feature+['Vt'],base_feature+["BET",'Vt'],\
#    base_feature+["BET","Vt","Vmic","Vmeso"],["BET","Vt"]+conditions,["BET","Vt","Vmic","Vmeso"]+conditions]
feature_list = [["BET","Vt","Vmic","Vmeso"]+base_feature]
columns = ['Gas','Model_name','CV_r2','CV_mse','test_r2_total_model','test_mse_by_total_model','test_r2_separa_model','test_mse_separa_model','best_param']
file_name = ["BET_Vt_Vmic_Vmeso"]#['BET_Curve',"BET_Curve_BET","BET_Cureve_Vt","BET_Curve_BET_Vt","BET_Curve_BET_Vt_Vmic_Vmeso","BET_Vt","BET_Vt_Vmic_Vmeso"] #['BET_Curve','BET_Plus_Vt',"Vmic_Vmeso"]
gas_list = ['CO2']#,'CFCs','Methane','E&E'b

for i in range(50):
    train_df,test_df = group_split(BET_df,random_state=i)
    """train_bet = train_df.loc[:,train_df.columns.str.startswith('BET_')]
    test_bet = test_df.loc[:,test_df.columns.str.startswith('BET_')]
    train_df.loc[:,train_df.columns.str.startswith('BET_')] = scaler.fit_transform(train_bet.values)
    test_df.loc[:,test_df.columns.str.startswith('BET_')] = scaler.transform(test_bet.values)"""
    
    for j in range(len(feature_list)):
        results = model_comparison(models,para_grids, feature_list[j],gas_list)
        files_name = 'Permutation_35_point_curve_feature_inverse_BET_CO2_'+file_name[j]+'_result_'+str(i)+'.csv'
        pd.DataFrame(results,columns = columns+feature_list[j]).to_csv(os.path.join('./8_using_BET_curve',files_name)) 
    print("The finish of {} th trial".format(i)) 

['BET_33', 'BET_32', 'BET_31', 'BET_30', 'BET_29', 'BET_28', 'BET_27', 'BET_26', 'BET_25', 'BET_24', 'BET_23', 'BET_22', 'BET_21', 'BET_20', 'BET_19', 'BET_18', 'BET_17', 'BET_16', 'BET_15', 'BET_14', 'BET_13', 'BET_12', 'BET_11', 'BET_10', 'BET_9', 'BET_8', 'BET_7', 'BET_6', 'BET_5', 'BET_4', 'BET_3', 'BET_2', 'BET_1', 'BET_0']
Dataset CO2, Algorithm LGBM_total, Test_r2 0.9447501100187128, Test_error 0.4764456226811341
The finish of 0 th trial
Dataset CO2, Algorithm LGBM_total, Test_r2 0.9534045831072581, Test_error 0.3544460456976484
The finish of 1 th trial
Dataset CO2, Algorithm LGBM_total, Test_r2 0.9437348961203718, Test_error 0.38214250076834066
The finish of 2 th trial
Dataset CO2, Algorithm LGBM_total, Test_r2 0.9357408554651855, Test_error 0.490093860605517
The finish of 3 th trial
Dataset CO2, Algorithm LGBM_total, Test_r2 0.9604635651627972, Test_error 0.6128655626757435
The finish of 4 th trial
Dataset CO2, Algorithm LGBM_total, Test_r2 0.8989208676400957, Test_error 1.072

KeyboardInterrupt: 

# Post result treatments
 

In [96]:
import pandas as pd
"""
file_name = ['BET_Curve',"BET_Curve_BET","BET_Cureve_Vt","BET_Curve_BET_Vt","BET_Curve_BET_Vt_Vmic_Vmeso","BET_Vt","BET_Vt_Vmic_Vmeso"] 
feature_list = [base_feature,base_feature+["BET"],base_feature+['Vt'],base_feature+["BET",'Vt'],\
    base_feature+["BET","Vt","Vmic","Vmeso"],["BET","Vt"]+conditions,["BET","Vt","Vmic","Vmeso"]+conditions]
"""
feature_list = [base_feature+["BET","Vt","Vmic","Vmeso"]]
columns = ['Gas','Model_name','CV_r2','CV_mse','test_r2_total_model','test_mse_by_total_model','test_r2_separa_model','test_mse_separa_model','best_param']
file_name = ["BET_Vt_Vmic_Vmeso"]
cal_columns= ["CV_r2","CV_mse","test_r2_separa_model","test_mse_separa_model"]
for j in range(len(file_name)):
    df_list = []
    for i in range(50):
        files_name = 'Permutation_35_point_curve_feature_inverse_BET_CO2_'+file_name[j]+'_result_'+str(i)+'.csv'
        temp_pd = pd.read_csv(os.path.join('./8_using_BET_curve',files_name))[cal_columns+feature_list[j]]
        temp_pd['feature_sum'] = temp_pd[feature_list[j]].sum(axis=1)
       

        for feature in feature_list[j]:
            temp_pd[feature]=temp_pd[feature]/temp_pd["feature_sum"]
        
        df_list.append(temp_pd)
        pd.concat(df_list).groupby(level=0).mean().to_csv(os.path.join('./8_using_BET_curve',file_name[j]+'_mean.csv'))
        pd.concat(df_list).groupby(level=0).std().to_csv(os.path.join('./8_using_BET_curve',file_name[j]+'_std.csv'))